In [38]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
train_dir = 'leapGestRecog'

# Prétraitement des images avec ImageDataGenerator
# Appliquer l'augmentation d'image pour éviter le sur-apprentissage
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normaliser les pixels entre 0 et 1
    shear_range=0.2,    # Appliquer des transformations aléatoires sur les images
    zoom_range=0.2,
    horizontal_flip=True
)

# Chargement des données d'entraînement
train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),  # Redimensionner les images
    batch_size=32,
    class_mode='categorical'  # Classification multiple (plusieurs classes)
)

# Obtenir les classes (gestes)
classes = list(train_ds.class_indices.keys())

Found 20000 images belonging to 10 classes.


## Construction du model 

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Définir le modèle CNN pour la reconnaissance des gestes
model = Sequential()

# Ajouter une couche de convolution avec des filtres
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # Aplatissement des sorties de la couche convolutive

# Ajouter des couches denses pour la classification
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Ajouter une régularisation Dropout pour éviter le sur-apprentissage
model.add(Dense(len(classes), activation='softmax'))  # Sortie avec une classe par geste


# Compilation du modèl 
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Resumer du modèl
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 684,490 (2.61 MB)

 Trainable params: 684,490 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

## Entrainement du modèl 

In [41]:
# Entraînement du modèle
history = model.fit(
    train_ds, 
    epochs=20,
    steps_per_epoch=train_ds.samples // train_ds.batch_size
)

Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 162s 257ms/step - accuracy: 0.3920 - loss: 1.6428
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 69s 110ms/step - accuracy: 0.8775 - loss: 0.3260
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 76s 121ms/step - accuracy: 0.9375 - loss: 0.1683
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 76s 122ms/step - accuracy: 0.9591 - loss: 0.1110
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 77s 123ms/step - accuracy: 0.9615 - loss: 0.0993
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 77s 123ms/step - accuracy: 0.9645 - loss: 0.0857
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 77s 123ms/step - accuracy: 0.9703 - loss: 0.0747
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 78s 125ms/step - accuracy: 0.9719 - loss: 0.0615
Epoch 9/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 78s 124ms/step - accuracy: 0.9733 - loss: 0.0670
Epoch 10/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 154s 247ms/step - accuracy: 0.9770 - loss: 0.0589
Epoch 11/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 79s 126ms/step - accuracy: 0.9792 - loss: 0.0523
Epoch 12/20
625/6

## Enregistrement du modèle 

In [45]:
# Sauvegarde du modèle
model.save('gesture_recognition_model.h5')


## Deployement 

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Charger le modèle pré-entraîné
model = tf.keras.models.load_model('gesture_recognition_model.h5')  # Remplacez par le chemin de votre modèle

# Définir la taille d'entrée des images (la même que celle utilisée lors de l'entraînement)
img_size = 64

# Initialiser la caméra (indice 0 pour la caméra par défaut)
cap = cv2.VideoCapture(0)

# Définir la liste des classes (gestes)
classes = ['palm', 'l', 'fist', 'fist_moved', 'thumb', 'index', 'ok', 'palm_moved', 'c', 'down']

while True:
    # Lire une image depuis la caméra
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Redimensionner l'image pour la compatibilité avec le modèle
    resized_frame = cv2.resize(frame, (img_size, img_size))

    # Normaliser les pixels de l'image
    input_frame = resized_frame / 255.0

    # Ajouter une dimension supplémentaire pour simuler un batch (le modèle attend un lot d'images)
    input_frame = np.expand_dims(input_frame, axis=0)

    # Faire la prédiction sur l'image
    predictions = model.predict(input_frame)

    # Obtenir l'indice de la classe avec la probabilité la plus élevée
    predicted_class_idx = np.argmax(predictions)

    # Obtenir le nom de la classe prédite
    predicted_class = classes[predicted_class_idx]

    # Afficher le résultat sur l'image
    cv2.putText(frame, f"Predicted: {predicted_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Afficher l'image avec le geste prédit
    cv2.imshow('Gesture Recognition', frame)

    # Quitter la boucle si la touche 'q' est pressée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━